# Import Packages

In [9]:
import pandas as pd
import numpy as np

# Load Data

In [10]:
input_df = pd.read_csv("./data/20240117_churn_data.csv", low_memory=False).drop_duplicates()

# Prepare Data

## Functions

In [12]:
def tryconvert(value, default, *types):
    for t in types:
        try:
            return t(value)
        except (ValueError, TypeError, IndexError):
            continue
    return default

def count_decreases(group):
    # Shift the 'Value' column down to compare with the next row
    prev_values = group['policy_nr_hashed'].shift(1)
    # Check if the current value is greater than the next value
    decreases = group['policy_nr_hashed'] > prev_values
    # Sum the True values (which represent decreases)
    return decreases.sum()

def last_non_zero(series):
    non_zero_values = series.replace(0, np.NaN).dropna()
    if not non_zero_values.empty:
        return non_zero_values.iloc[-1]
    else:
        return 0

## Filter Customers

In [13]:
# Only keep customers that joined after welcome discount was introduced
input_df = input_df[input_df["year_initiation_policy"] >= 2021].sort_values("year_initiation_policy_version")

# We need to filter out all the customers that churned 
input_df['non_relevant_churn'] = input_df.groupby('policy_nr_hashed')[['d_churn_between_prolongations', 'd_churn_cancellation']].transform('sum').sum(axis=1)
input_df = input_df[input_df['non_relevant_churn'] == 0]

## Create Relevant Columns

In [14]:
# A variable that says how long it took for a customer to churn (0 means he didn't churn yet, 5 means he churned 5 years after joining)
input_df["years_to_churn"] = input_df["years_since_policy_started"] * input_df["d_churn_around_prolongation"]

# A column that says whether a customer got a discount or not
input_df["has_discount"] = (input_df.groupby('policy_nr_hashed')[['welcome_discount']].transform('min') < 1)

# Create a dict of all the premium data
input_df['premium_data'] = input_df.apply(lambda row: {'year': int(row['year_initiation_policy_version']), 'main': np.round(row['premium_main_coverages'], 0), 'supp': np.round(row['premium_supplementary_coverages'], 0), 'total': np.round(row['total_premium'], 0)}, axis=1)

# How much of the premium is main coverage
input_df['main_coverage_portion'] = input_df['premium_main_coverages'] / input_df['total_premium']

# Aggregate all car-data into a single column
input_df['car_data'] = input_df["brand"] + input_df["type"] + input_df["weight"].astype(str) + input_df["fuel_type"]
input_df["lagged_car_data"] = input_df.groupby('policy_nr_hashed')['car_data'].shift(1)
input_df["car_change"] = ((input_df["lagged_car_data"] != input_df["car_data"]) & ~input_df['lagged_car_data'].isnull())
input_df["year_car_change"] = input_df["car_change"].astype("int") * input_df["year_initiation_policy_version"]

# Concatenate all mutations
input_df["all_mutations"] = input_df[[col for col in input_df.columns if (("mutation" in col) and (len(col) < 12))]].astype("str").sum(1).str.replace('nan', '')

# Tag when a policy has changed holder
input_df["policyholder_change"] = input_df["all_mutations"].str.contains("replacePolicyholder")
input_df["fake_alarm"] = input_df["all_mutations"].str.contains("restoreCancellation")

# Compute total lagged coverage
input_df['n_coverages_trend'] = input_df["n_coverages"] - input_df.groupby('policy_nr_hashed')['n_coverages'].shift(1)

# Calculate number of accdent years
input_df["accident_years"] = ((input_df.groupby('policy_nr_hashed')['accident_free_years'].shift(1) > input_df['accident_free_years']).astype("int") * (input_df.groupby('policy_nr_hashed')['accident_free_years'].shift(1) - input_df['accident_free_years'])).fillna(0).replace(-0.0, 0)

# Create lagged premium difference (abs and perc)
input_df["lagged_total_premium"] = input_df.groupby('policy_nr_hashed')['total_premium'].shift(1)
input_df["abs_diff_total_premium"] = input_df["total_premium"] - input_df["lagged_total_premium"]
input_df["perc_diff_total_premium"] = input_df["abs_diff_total_premium"] / input_df["lagged_total_premium"]

In [15]:
customer_data_columns = ['customer_age', 'accident_free_years', 'car_value', 'age_car', 'brand', 'type', 'weight', 'fuel_type', 'postcode', 'product', 'allrisk basis', 'allrisk compleet', 'allrisk royaal', 'wa-extra', 'wettelijke aansprakelijkheid']
customer_data_agg = {f'last_{col}': pd.NamedAgg(column=col, aggfunc='last') for col in customer_data_columns}

In [16]:
final_df = (
    input_df
    .sort_values("year_initiation_policy_version")
    .groupby("policy_nr_hashed")
    .agg(
        welcome_discount=pd.NamedAgg(column="welcome_discount", aggfunc="min"),
        last_data_year=pd.NamedAgg(column="year_initiation_policy_version", aggfunc="last"),
        years_to_churn=pd.NamedAgg(column="years_to_churn", aggfunc="max"),
        control_group=pd.NamedAgg(column="welcome_discount_control_group", aggfunc="first"),
        premiums=pd.NamedAgg(column="premium_data", aggfunc=lambda x: x.to_list()),
        first_premium=pd.NamedAgg(column="total_premium", aggfunc='first'),
        last_premium=pd.NamedAgg(column="total_premium", aggfunc='last'),
        first_split=pd.NamedAgg(column="main_coverage_portion", aggfunc='first'),
        last_split=pd.NamedAgg(column="main_coverage_portion", aggfunc='last'),
        **customer_data_agg,
        nr_cars=pd.NamedAgg(column="car_data", aggfunc=lambda x:np.size(set(x.to_list()))),
        fake_alarm=pd.NamedAgg(column="fake_alarm", aggfunc='sum'),
        policyholder_change=pd.NamedAgg(column="policyholder_change", aggfunc='sum'),
        max_nr_coverages=pd.NamedAgg(column="n_coverages", aggfunc='max'),
        last_nr_coverages=pd.NamedAgg(column="n_coverages", aggfunc='last'),
        last_trend_nr_coverages=pd.NamedAgg(column="n_coverages_trend", aggfunc="last"),
        accident_years=pd.NamedAgg(column="accident_years", aggfunc="sum"),
        last_year_car_change=pd.NamedAgg(column="year_car_change", aggfunc=last_non_zero),
        last_change_premium_abs=pd.NamedAgg(column="abs_diff_total_premium", aggfunc="last"),
        last_change_premium_perc=pd.NamedAgg(column="perc_diff_total_premium", aggfunc="last"),
    )
    .reset_index()
)

In [17]:
final_df["years_since_last_car_change"] = (final_df["last_data_year"] - final_df["last_year_car_change"]).astype("int").apply(lambda x: x if x <= 10 else np.NaN)
final_df["n_last_vs_peak"] = final_df["last_nr_coverages"] - final_df["max_nr_coverages"]
final_df["last_vs_first_split"] = final_df["last_split"] - final_df["first_split"]
final_df["lpa"] = (~final_df["control_group"].str.contains("no LPA")).astype("int")
final_df["cum_change_premium_abs"] = final_df["last_premium"] - final_df["first_premium"]
final_df["cum_change_premium_perc"] = final_df["cum_change_premium_abs"] / final_df["first_premium"]

# final_df.to_csv("./data/prepped_data.csv")